![](https://www.pata.org/wp-content/uploads/2014/09/TripAdvisor_Logo-300x119.png)
# Predict TripAdvisor Rating
## В этом соревновании нам предстоит предсказать рейтинг ресторана в TripAdvisor
**По ходу задачи:**
* Прокачаем работу с pandas
* Научимся работать с Kaggle Notebooks
* Поймем как делать предобработку различных данных
* Научимся работать с пропущенными данными (Nan)
* Познакомимся с различными видами кодирования признаков
* Немного попробуем [Feature Engineering](https://ru.wikipedia.org/wiki/Конструирование_признаков) (генерировать новые признаки)
* И совсем немного затронем ML
* И многое другое...   



### И самое важное, все это вы сможете сделать самостоятельно!

*Этот Ноутбук являетсся Примером/Шаблоном к этому соревнованию (Baseline) и не служит готовым решением!*   
Вы можете использовать его как основу для построения своего решения.

> что такое baseline решение, зачем оно нужно и почему предоставлять baseline к соревнованию стало важным стандартом на kaggle и других площадках.   
**baseline** создается больше как шаблон, где можно посмотреть как происходит обращение с входящими данными и что нужно получить на выходе. При этом МЛ начинка может быть достаточно простой, просто для примера. Это помогает быстрее приступить к самому МЛ, а не тратить ценное время на чисто инженерные задачи. 
Также baseline являеться хорошей опорной точкой по метрике. Если твое решение хуже baseline - ты явно делаешь что-то не то и стоит попробовать другой путь) 

В контексте нашего соревнования baseline идет с небольшими примерами того, что можно делать с данными, и с инструкцией, что делать дальше, чтобы улучшить результат.  Вообще готовым решением это сложно назвать, так как используются всего 2 самых простых признака (а остальные исключаются).

# import

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt
import seaborn as sns 
%matplotlib inline
import re

# Загружаем специальный удобный инструмент для разделения датасета:
from sklearn.model_selection import train_test_split

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
# всегда фиксируйте RANDOM_SEED, чтобы ваши эксперименты были воспроизводимы!
RANDOM_SEED = 42

In [ ]:
# зафиксируем версию пакетов, чтобы эксперименты были воспроизводимы:
!pip freeze > requirements.txt

# DATA

In [ ]:
DATA_DIR = '/kaggle/input/sf-dst-restaurant-rating/'
df_train = pd.read_csv(DATA_DIR+'/main_task.csv')
df_test = pd.read_csv(DATA_DIR+'kaggle_task.csv')
sample_submission = pd.read_csv(DATA_DIR+'/sample_submission.csv')

In [ ]:
df_train.info()

In [ ]:
df_train.head(5)

In [ ]:
df_test.info()

In [ ]:
df_test.head(5)

In [ ]:
sample_submission.head(5)

In [ ]:
sample_submission.info()

In [ ]:
# ВАЖНО! дря корректной обработки признаков объединяем трейн и тест в один датасет
df_train['sample'] = 1 # помечаем где у нас трейн
df_test['sample'] = 0 # помечаем где у нас тест
df_test['Rating'] = 0 # в тесте у нас нет значения Rating, мы его должны предсказать, по этому пока просто заполняем нулями

data = df_test.append(df_train, sort=False).reset_index(drop=True) # объединяем

In [ ]:
data.info()

Подробнее по признакам:
* City: Город 
* Cuisine Style: Кухня
* Ranking: Ранг ресторана относительно других ресторанов в этом городе
* Price Range: Цены в ресторане в 3 категориях
* Number of Reviews: Количество отзывов
* Reviews: 2 последних отзыва и даты этих отзывов
* URL_TA: страница ресторана на 'www.tripadvisor.com' 
* ID_TA: ID ресторана в TripAdvisor
* Rating: Рейтинг ресторана

In [ ]:
data.sample(5)

In [ ]:
# Переименуем для удобства колонки 
data.columns = ["restaurant_id", "city", "cuisine", "ranking", "price", "number_of_reviews",
              "reviews", "URL", "ID", "sample", "Rating"]

In [ ]:
data.isna().sum()

In [ ]:
# Первоначальный просмотр общих значений.
def viewing(column):
    print('') 
    print('Столбец', column)
    display(data[column].value_counts())
    print('Количество пропусков: {}'.format(data[column].isnull().sum()), 
          'Количество уникальных значений: {}'.format(data[column].nunique()), sep='\n')

In [ ]:
# создадим функцию для получения интересующих нас параметров
def quantity_check(column):
    ab_25perc = data[column].quantile(0.25, interpolation="midpoint")
    ab_75perc = data[column].quantile(0.75, interpolation="midpoint")
    ab_IQR = ab_75perc-ab_25perc
    print('Q1:{}'.format(ab_25perc), 'Q3:{}'.format(ab_75perc), 'IQR:{}'.format(ab_IQR),
          'Граница выбросов: [{a},{b}]'.format(a=ab_25perc-1.5*ab_IQR, b=ab_75perc+1.5*ab_IQR), sep='\n')

In [ ]:
viewing('restaurant_id')

Посмотрим, какие рестораны сетевые, а какие нет. И заполним 1 или 0.

In [ ]:
# Создадим переменную для сетевых ресторанов.
chain = list(data.restaurant_id.value_counts()[data.restaurant_id.value_counts() > 1].index)

In [ ]:
# Создадим новую колонку для наших ресторанов.
data['chain_restaurant'] = data[data.restaurant_id.isin(chain)].restaurant_id.apply(lambda x: 1)
data['chain_restaurant'].fillna(0, inplace=True)

In [ ]:
viewing('city')

In [ ]:
plt.figure(figsize=(15, 5))
sns.boxplot(x='city', y='Rating', data=data[data['sample'] == 1])
plt.xticks(rotation=90);

In [ ]:
df_train['City'].value_counts(ascending=True).plot(kind='barh')

Видим что выделяются 4 города: Милан, Рим, Лондон, Париж.
Милан - рейтинг до 4.5. У остальных до 5.0 Милан где-то не дотягивает. Странно...
Рим - рейтинг от 3.0. Значит, что Кухня людей устраивает. Можно туда ехать и не бояться, что что-то будет не так.)))
Лондон и Париж - два города с самым большим кол-вом ресторанов. Но в Лондоне всё-таки их больше.

In [ ]:
# Заменяем ошибочное название города Порту на правильное. По Копенгагену и Мюнхену вопрос...
# но в целом принять можно.
data.loc[data.city == 'Oporto', 'city'] = 'Porto'

In [ ]:
# Население городов
population = {
            'Paris': 2148327,
            'Stockholm': 961609,
            'London': 8908081,
            'Berlin': 3644826,
            'Munich': 1471508,
            'Porto': 237591,
            'Milan': 1378689,
            'Bratislava': 437725,
            'Vienna': 1897491,
            'Rome': 2870500,
            'Barcelona': 1636762,
            'Madrid': 3266126,
            'Dublin': 1173179,
            'Brussels': 179277,
            'Zurich': 428737,
            'Warsaw': 1790658,
            'Budapest': 1752286,
            'Copenhagen': 615993,
            'Amsterdam': 872757,
            'Lyon': 506615,
            'Hamburg': 1841179,
            'Lisbon': 505526,
            'Prague': 1301132,
            'Oslo': 673469,
            'Helsinki': 655281,
            'Edinburgh': 488100,
            'Geneva': 200548,
            'Ljubljana': 284355,
            'Athens': 664046,
            'Luxembourg': 115227,
            'Krakow': 779115
        }
data['population'] = data['city'].map(population)

In [ ]:
#Количество ресторанов в городе
rest_count_dict = dict(data['city'].value_counts())
data['rest_count'] = data['city'].map(rest_count_dict)

In [ ]:
# Относительный рейтинг по городу
data['ranking_per_city'] = data['ranking'] / data['rest_count']

In [ ]:
# Задаём столицы и выводим 1 или 0
Capitals = ['London', 'Paris', 'Madrid', 'Berlin', 'Rome', 'Prague', 'Lisbon', 'Vienna', 'Amsterdam', 'Brussels', 'Stockholm',
            'Budapest', 'Warsaw', 'Dublin', 'Copenhagen', 'Athens', 'Edinburgh', 'Oslo', 'Helsinki', 'Bratislava', 'Luxemburg', 'Ljubljana']
data['capital'] = data['city'].apply(lambda x: 1 if x in Capitals else 0)

In [ ]:
data['not_capital'] = data['capital'].apply(lambda x: 1 if x == 0 else 0)

In [ ]:
# Площадь городов (кв.км)
square_area = {'Amsterdam': 219, 'Athens': 412, 'Barcelona': 101, 'Berlin': 891, 'Bratislava': 368, 'Brussels': 32.6,
        'Budapest': 525, 'Copenhagen': 86.4, 'Dublin': 115, 'Edinburgh': 175, 'Geneva': 15.9, 'Hamburg': 755,
        'Helsinki': 715, 'Krakow': 327, 'Lisbon': 100, 'Ljubljana': 163, 'London': 1706, 'Luxembourg': 51.5,
        'Lyon': 47.9, 'Madrid': 607, 'Milan': 181, 'Munich': 310, 'Porto': 41.7, 'Oslo': 454, 'Paris': 105,
        'Prague': 496, 'Rome': 1287, 'Stockholm': 188, 'Vienna': 414, 'Warsaw': 517, 'Zurich': 91.9}
data['square_area'] = data['city'].map(square_area)

Создали 3 новые колонки на основе информации о городах.

In [ ]:
viewing('cuisine')

Поработаем с кухнями и выберем 50 самых непопулярных и 50 самых популярных. Заменим пропуски на "Other".

Для этого создадим 2 переменные и 2 функции, а также на основании этого 2 новые колонки.

In [ ]:
data['cuisine'] = data['cuisine'].apply(lambda x: re.findall(r"'(\b.*?\b)'", str(x)))

In [ ]:
cuisine_low_lst = data.explode('cuisine')['cuisine'].value_counts()[
    data.explode('cuisine')['cuisine'].value_counts() < 50].index.tolist()

In [ ]:
def cuisine_low_count(cell):
    x = 0
    for i in cuisine_low_lst:
        if i in cell:
            x += 1
    return x

In [ ]:
data['low_cuisine'] = data['cuisine'].apply(cuisine_low_count)

In [ ]:
cuisine_high_lst = data.explode('cuisine')['cuisine'].value_counts()[
    data.explode('cuisine')['cuisine'].value_counts() > 50].index.tolist()

In [ ]:
def cuisine_high_count(cell):
    x = 0
    for i in cuisine_high_lst:
        if i in cell:
            x += 1
    return x

In [ ]:
data['high_cuisine'] = data['cuisine'].apply(cuisine_high_count)

In [ ]:
data['cuisine'].fillna("['Other']", inplace=True)

Посмотрим кол-во кухонь и сделаем топ 10 кухонь по версии моей мамы))) И оформим в 10 новых колонок + 1.

In [ ]:
Cuisine_x = data['cuisine']
Cuisine_number = Cuisine_x.explode() 

Cuisine_number.value_counts().head(50).plot(kind='bar', title='cuisine')
Cuisine_number.value_counts().head(50)

В колонке есть данные, не соответствующие этой колонке (на мой взгляд), например Cafe, Pub и т.д. (это же заведение, а не кухня (может конечно подразумеваться кухня в этих заведениях?), тем не менее это значение вызывает много вопросов), а удалить строку по условию нельзя, только колонку. Поэтому сложно принять решение что нужно, при составлении отдельных топ 10 колонок. 

In [ ]:
# 10 топовых типов кухонь
def c0(rec):
    if type(rec)==float:
        return 0
    elif 'Vegetarian Friendly' in rec or 'Vegan Options' in rec or 'Gluten Free Options'in rec:
        return 1
    else:
        return 0
data['Vegetarian']=data['cuisine'].apply(c0)

def c1(rec):
    if type(rec)==float:
        return 0
    elif 'European' in rec:
        return 1
    else:
        return 0
data['European']=data['cuisine'].apply(c1)

def c2(rec):
    if type(rec)==float:
        return 0
    elif 'Mediterranean' in rec:
        return 1
    else:
        return 0
data['Mediterranean']=data['cuisine'].apply(c2)

def c3(rec):
    if type(rec)==float:
        return 0
    elif 'Italian' in rec or 'Pizza' in rec:
        return 1
    else:
        return 0
data['Italian']=data['cuisine'].apply(c3)

def c4(rec):
    if type(rec)==float:
        return 0
    elif 'Pub' in rec or 'Bar' in rec or 'Wine Bar' in rec or 'Cafe' in rec:
        return 1
    else:
        return 0
data['Bar']=data['cuisine'].apply(c4)

def c5(rec):
    if type(rec)==float:
        return 0
    elif 'French' in rec:
        return 1
    else:
        return 0
data['French']=data['cuisine'].apply(c5)

def c6(rec):
    if type(rec)==float:
        return 0
    elif 'Asian' in rec:
        return 1
    else:
        return 0
data['Asian']=data['cuisine'].apply(c6)

def c7(rec):
    if type(rec)==float:
        return 0
    elif 'Spanish' in rec:
        return 1
    else:
        return 0
data['Spanish']=data['cuisine'].apply(c7)

def c8(rec):
    if type(rec)==float:
        return 0
    elif 'American' in rec or 'Fast Food' in rec:
        return 1
    else:
        return 0
data['American']=data['cuisine'].apply(c8)

def c9(rec):
    if type(rec)==float:
        return 0
    elif 'Japanese' in rec or 'Seafood' in rec or 'Sushi' in rec:
        return 1
    else:
        return 0
data['Japanese']=data['cuisine'].apply(c9)

def c10(rec):
    if type(rec)==float:
        return 0
    elif 'Vegetarian Friendly' in rec or 'Vegan Options' in rec or 'Gluten Free Options' in rec or 'Japanese' in rec or 'Seafood' in rec or 'Sushi' in rec or 'American' in rec or 'Fast Food' in rec or 'Spanish' in rec or 'Asian' in rec or 'French' in rec or 'Pub' in rec or 'Bar' in rec or 'Wine Bar' in rec or 'Italian' in rec or 'Pizza' in rec or 'Mediterranean' in rec or 'European' in rec:
        return 0
    else:
        return 1
data['Other_cuisine']=data['cuisine'].apply(c10)

In [ ]:
viewing('ranking')

In [ ]:
# посмотрим на топ 10 городов
for x in (data['city'].value_counts())[0:10].index:
    data['ranking'][data['city'] == x].hist(bins=35)

Получается, что Ranking имеет нормальное распределение, просто в больших городах больше ресторанов, из-за этого мы имеем смещение. Разделим Ranking на количество ресторанов в городе.

Создадим минимальный, средний и максимальный ранг для городов. И запишем их в новые колонки.

In [ ]:
# Минимальный ранг по городу 
min_city_ranking = dict(data.groupby(['city'])['ranking'].min())
data['Min_ranking'] = data['city'].map(min_city_ranking)

In [ ]:
# Средний ранг по городу 
mean_city_ranking = dict(data.groupby(['city'])['ranking'].mean())
data['Mean_ranking'] = data['city'].map(mean_city_ranking)

In [ ]:
# Максимальный ранг по городу 
max_city_ranking = dict(data.groupby(['city'])['ranking'].max())
data['Max_ranking'] = data['city'].map(max_city_ranking)

In [ ]:
viewing('Rating')

In [ ]:
viewing('price')

In [ ]:
# Пропуски можем заполнить самым популярным значением.
data['price'] = data['price'].fillna('$$ - $$$')

# Меняем значения на цифровой вариант.
replace_price_range = {'$': 1, '$$ - $$$': 2, '$$$$':4}
data['price'] = data['price'].map(replace_price_range)

Хотя из-за того, что пропущенных значений много, можно было бы и подумать как их можно ещё распределить. Но мыслей пока нет, может потом появятся. Должны появиться)))

In [ ]:
viewing('number_of_reviews')

In [ ]:
data['Number_of_Reviews_isNAN'] = pd.isna(data['number_of_reviews']).astype('uint8')

In [ ]:
# Заполним пропущенные значения средним по городу.
data['number_of_reviews'] = data.groupby("city")['number_of_reviews'].transform(
    lambda x: x.fillna(x.mean()))

In [ ]:
viewing('reviews')

Колонку reviews с отзывами я удалю. Почему? По 2-3 отзыва с разбросанными датами...этого очень мало. Надо парсить отзывы, но я к сожалению пока это не проходил, а времени на изучение парсинга нет((( Поэтому это всё в другой раз))) Т.е. в следующий заход)

In [ ]:
viewing('URL')

В данной колонке вижу только названия ресторанов. Всё остальное не выглядит как полезные данные.

In [ ]:
# Выносим названия ресторанов из колонки URL
data['URL'] = data['URL'].str.split('-')
# Приводим к единому регистру.
data['URL'] = [x[4].lower() for x in data['URL']]

In [ ]:
# Посмотрим кол-во ресторанов и выведим 50 из них.
Cuisine_x = data['URL']
Cuisine_number = Cuisine_x.explode() 

Cuisine_number.value_counts().head(50).plot(kind='bar', title='Restaurans')
Cuisine_number.value_counts().head(50)

Макдональдс больше всех в разы. Совпадение? Не думаю.

In [ ]:
# Возьмём рестораны, где их кол-во более 2. И заполним 1 и 0.
URL_limit = 2
URL_cnts = Cuisine_number.value_counts()
URL_columns = list(URL_cnts[URL_cnts > URL_limit].index)
Other_columns = list(set(URL_cnts.index) - set(URL_columns))
    
# И соответственно заменим на 1 или 0.
for URL in URL_columns:
    data[URL] = data['URL'].astype(
            'str').apply(lambda x: 1 if URL in x else 0)


def other_URL(URL_str):
    for URL in Other_columns:
        if URL in URL_str:
            return 1
    return 0

In [ ]:
viewing('ID')

Т.к. колонки restaurant_id и ID одинаковы. Забегая наперёд, restaurant_id мы удалим, а из ID достанем идентификатор.

In [ ]:
data['ID'] = data['ID'].apply(lambda x: int(x[1:]))

**Посмотрим корреляцию**

In [ ]:
plt.rcParams['figure.figsize'] = (15,10)
sns.heatmap(data.drop(['sample'], axis=1).corr(),)

In [ ]:
# Удаляем ненужные колонки.
data=data.drop(['restaurant_id', 'reviews'], axis=1)

In [ ]:
# для One-Hot Encoding в pandas есть готовая функция - get_dummies.
data = pd.get_dummies(data, columns=['city'], dummy_na=True)

Если так, то почему бы и не применить)))

# Data Preprocessing
Теперь, для удобства и воспроизводимости кода, завернем всю обработку в одну большую функцию.

In [ ]:
# на всякий случай, заново подгружаем данные

df_train = pd.read_csv(DATA_DIR+'/main_task.csv')
df_test = pd.read_csv(DATA_DIR+'/kaggle_task.csv')
df_train['sample'] = 1 # помечаем где у нас трейн
df_test['sample'] = 0 # помечаем где у нас тест
df_test['Rating'] = 0 # в тесте у нас нет значения Rating, мы его должны предсказать, по этому пока просто заполняем нулями

data = df_test.append(df_train, sort=False).reset_index(drop=True) # объединяем
data.info()

In [ ]:
def preproc_data(df_input):
    '''includes several functions to pre-process the predictor data.'''
    
    df_output = df_input.copy()
    
    # ################### 1. Предобработка ############################################################## 
    
    
    # 
    def viewing(column):
        print('') 
        print('Столбец', column)
        display(df_output[column].value_counts())
        print('Количество пропусков: {}'.format(df_output[column].isnull().sum()), 
              'Количество уникальных значений: {}'.format(df_output[column].nunique()), sep='\n')
        
    # 
    def quantity_check(column):
        ab_25perc = data[column].quantile(0.25, interpolation="midpoint")
        ab_75perc = data[column].quantile(0.75, interpolation="midpoint")
        ab_IQR = ab_75perc-ab_25perc
        print('Q1:{}'.format(ab_25perc), 'Q3:{}'.format(ab_75perc), 'IQR:{}'.format(ab_IQR),
              'Граница выбросов: [{a},{b}]'.format(a=ab_25perc-1.5*ab_IQR, b=ab_75perc+1.5*ab_IQR), sep='\n')    

        
    #  
    df_output.columns = ["restaurant_id", "city", "cuisine", "ranking", "price", "number_of_reviews",
                  "reviews", "URL", "ID", "sample", "Rating"]
    
    # 
    chain = list(df_output.restaurant_id.value_counts()[df_output.restaurant_id.value_counts() > 1].index)
    # 
    df_output['chain_restaurant'] = df_output[df_output.restaurant_id.isin(chain)].restaurant_id.apply(lambda x: 1)
    df_output['chain_restaurant'].fillna(0, inplace=True)
    
    
    
    # 
    df_output.loc[df_output.city == 'Oporto', 'city'] = 'Porto'
    
    # 
    population = {
            'Paris': 2148327,
            'Stockholm': 961609,
            'London': 8908081,
            'Berlin': 3644826,
            'Munich': 1471508,
            'Porto': 237591,
            'Milan': 1378689,
            'Bratislava': 437725,
            'Vienna': 1897491,
            'Rome': 2870500,
            'Barcelona': 1636762,
            'Madrid': 3266126,
            'Dublin': 1173179,
            'Brussels': 179277,
            'Zurich': 428737,
            'Warsaw': 1790658,
            'Budapest': 1752286,
            'Copenhagen': 615993,
            'Amsterdam': 872757,
            'Lyon': 506615,
            'Hamburg': 1841179,
            'Lisbon': 505526,
            'Prague': 1301132,
            'Oslo': 673469,
            'Helsinki': 655281,
            'Edinburgh': 488100,
            'Geneva': 200548,
            'Ljubljana': 284355,
            'Athens': 664046,
            'Luxembourg': 115227,
            'Krakow': 779115
        }
    df_output['population'] = df_output['city'].map(population)
    
    
    #
    rest_count_dict = dict(df_output['city'].value_counts())
    df_output['rest_count'] = df_output['city'].map(rest_count_dict)
    
    # 
    df_output['ranking_per_city'] = df_output['ranking'] / df_output['rest_count']
    
    
    # 
    Capitals = ['London', 'Paris', 'Madrid', 'Berlin', 'Rome', 'Prague', 'Lisbon', 'Vienna', 'Amsterdam', 'Brussels', 'Stockholm',
                'Budapest', 'Warsaw', 'Dublin', 'Copenhagen', 'Athens', 'Edinburgh', 'Oslo', 'Helsinki', 'Bratislava', 'Luxemburg', 'Ljubljana']
    df_output['capital'] = df_output['city'].apply(lambda x: 1 if x in Capitals else 0)
    
    df_output['not_capital'] = df_output['capital'].apply(lambda x: 1 if x == 0 else 0)
    
    
    # 
    square_area = {'Amsterdam': 219, 'Athens': 412, 'Barcelona': 101, 'Berlin': 891, 'Bratislava': 368, 'Brussels': 33,
            'Budapest': 525, 'Copenhagen': 87, 'Dublin': 115, 'Edinburgh': 118, 'Geneva': 16, 'Hamburg': 755,
            'Helsinki': 715, 'Krakow': 327, 'Lisbon': 100, 'Ljubljana': 163, 'London': 1706, 'Luxembourg': 52,
            'Lyon': 48, 'Madrid': 607, 'Milan': 181, 'Munich': 310, 'Porto': 42, 'Oslo': 454, 'Paris': 105,
            'Prague': 496, 'Rome': 1287, 'Stockholm': 188, 'Vienna': 414, 'Warsaw': 517, 'Zurich': 92}
    df_output['square_area'] = df_output['city'].map(square_area)
    
    
    
    df_output['cuisine'] = df_output['cuisine'].apply(lambda x: re.findall(r"'(\b.*?\b)'", str(x)))
    df_output['Cuisine_Style_isNAN'] = pd.isna(df_output['cuisine']).astype('uint8')
    cuisine_low_lst = df_output.explode('cuisine')['cuisine'].value_counts()[
    df_output.explode('cuisine')['cuisine'].value_counts() < 50].index.tolist()
    
    def cuisine_low_count(cell):
        x = 0
        for i in cuisine_low_lst:
            if i in cell:
                x += 1
        return x

    df_output['low_cuisine'] = df_output['cuisine'].apply(cuisine_low_count)
    
    cuisine_high_lst = df_output.explode('cuisine')['cuisine'].value_counts()[
    df_output.explode('cuisine')['cuisine'].value_counts() > 50].index.tolist()
    
    def cuisine_high_count(cell):
        x = 0
        for i in cuisine_high_lst:
            if i in cell:
                x += 1
        return x

    df_output['high_cuisine'] = df_output['cuisine'].apply(cuisine_high_count)
    
    df_output['cuisine'].fillna("['Other']", inplace=True) 
    

        # 
    def c0(rec):
        if type(rec)==float:
            return 0
        elif 'Vegetarian Friendly' in rec or 'Vegan Options' in rec or 'Gluten Free Options'in rec:
            return 1
        else:
            return 0
    df_output['Vegetarian']=df_output['cuisine'].apply(c0)

    def c1(rec):
        if type(rec)==float:
            return 0
        elif 'European' in rec:
            return 1
        else:
            return 0
    df_output['European']=df_output['cuisine'].apply(c1)

    def c2(rec):
        if type(rec)==float:
            return 0
        elif 'Mediterranean' in rec:
            return 1
        else:
            return 0
    df_output['Mediterranean']=df_output['cuisine'].apply(c2)

    def c3(rec):
        if type(rec)==float:
            return 0
        elif 'Italian' in rec or 'Pizza' in rec:
            return 1
        else:
            return 0
    df_output['Italian']=df_output['cuisine'].apply(c3)

    def c4(rec):
        if type(rec)==float:
            return 0
        elif 'Pub' in rec or 'Bar' in rec or 'Wine Bar' in rec or 'Cafe' in rec:
            return 1
        else:
            return 0
    df_output['Bar']=df_output['cuisine'].apply(c4)

    def c5(rec):
        if type(rec)==float:
            return 0
        elif 'French' in rec:
            return 1
        else:
            return 0
    df_output['French']=df_output['cuisine'].apply(c5)

    def c6(rec):
        if type(rec)==float:
            return 0
        elif 'Asian' in rec:
            return 1
        else:
            return 0
    df_output['Asian']=df_output['cuisine'].apply(c6)

    def c7(rec):
        if type(rec)==float:
            return 0
        elif 'Spanish' in rec:
            return 1
        else:
            return 0
    df_output['Spanish']=df_output['cuisine'].apply(c7)

    def c8(rec):
        if type(rec)==float:
            return 0
        elif 'American' in rec or 'Fast Food' in rec:
            return 1
        else:
            return 0
    df_output['American']=df_output['cuisine'].apply(c8)

    def c9(rec):
        if type(rec)==float:
            return 0
        elif 'Japanese' in rec or 'Seafood' in rec or 'Sushi' in rec:
            return 1
        else:
            return 0
    df_output['Japanese']=df_output['cuisine'].apply(c9)

    def c10(rec):
        if type(rec)==float:
            return 0
        elif 'Vegetarian Friendly' in rec or 'Vegan Options' in rec or 'Gluten Free Options' in rec or 'Japanese' in rec or 'Seafood' in rec or 'Sushi' in rec or 'American' in rec or 'Fast Food' in rec or 'Spanish' in rec or 'Asian' in rec or 'French' in rec or 'Pub' in rec or 'Bar' in rec or 'Wine Bar' in rec or 'Italian' in rec or 'Pizza' in rec or 'Mediterranean' in rec or 'European' in rec:
            return 0
        else:
            return 1
    df_output['Other_cuisine']=df_output['cuisine'].apply(c10)

    
        
    # 
    min_city_ranking = dict(df_output.groupby(['city'])['ranking'].min())
    df_output['Min_ranking'] = df_output['city'].map(min_city_ranking)
    
    # 
    mean_city_ranking = dict(df_output.groupby(['city'])['ranking'].mean())
    df_output['Mean_ranking'] = df_output['city'].map(mean_city_ranking)
    
    # 
    max_city_ranking = dict(df_output.groupby(['city'])['ranking'].max())
    df_output['Max_ranking'] = df_output['city'].map(max_city_ranking)
    
    
    # 
    df_output['price'] = df_output['price'].fillna('$$ - $$$')

    # 
    replace_price_range = {'$': 1, '$$ - $$$': 2, '$$$$':4}
    df_output['price'] = df_output['price'].map(replace_price_range)
    
    
    df_output['Number_of_Reviews_isNAN'] = pd.isna(df_output['number_of_reviews']).astype('uint8')
    
    # 
    df_output['number_of_reviews'] = df_output.groupby("city")['number_of_reviews'].transform(
        lambda x: x.fillna(x.mean()))
    
    
    
    Cuisine_x = df_output['URL']
    Cuisine_number = Cuisine_x.explode()
    
    
    # 
    URL_limit = 2
    URL_cnts = Cuisine_number.value_counts()
    URL_columns = list(URL_cnts[URL_cnts > URL_limit].index)
    Other_columns = list(set(URL_cnts.index) - set(URL_columns))
    
    
    for URL in URL_columns:
        df_output[URL] = df_output['URL'].astype(
            'str').apply(lambda x: 1 if URL in x else 0)


    def other_URL(URL_str):
        for URL in Other_columns:
            if URL in URL_str:
                return 1
        return 0

    

    df_output['ID'] = df_output['ID'].apply(lambda x: int(x[1:]))
    
    
    # ################### 3. Encoding ############################################################## 
    df_output = pd.get_dummies(df_output, columns=['city'], dummy_na=True)
    
    
    
    # ################### 5. Clean #################################################### 
    # убираем признаки которые еще не успели обработать, 
    # модель на признаках с dtypes "object" обучаться не будет, просто выберим их и удалим
    object_columns = [s for s in df_output.columns if df_output[s].dtypes == 'object']
    df_output.drop(object_columns, axis = 1, inplace=True)
    
    return df_output

>По хорошему, можно было бы перевести эту большую функцию в класс и разбить на подфункции (согласно ООП). Но по моему опыту я только слышал про это, но пока это не проходили, далее в обучении освою ООП)))

#### Запускаем и проверяем что получилось

In [ ]:
df_preproc = preproc_data(data)
df_preproc.sample(10)

In [ ]:
df_preproc.info()

In [ ]:
# Теперь выделим тестовую часть
train_data = df_preproc.query('sample == 1').drop(['sample'], axis=1)
test_data = df_preproc.query('sample == 0').drop(['sample'], axis=1)

y = train_data.Rating.values            # наш таргет
X = train_data.drop(['Rating'], axis=1)

**Перед тем как отправлять наши данные на обучение, разделим данные на еще один тест и трейн, для валидации. 
Это поможет нам проверить, как хорошо наша модель работает, до отправки submissiona на kaggle.**

In [ ]:
# Воспользуемся специальной функцией train_test_split для разбивки тестовых данных
# выделим 20% данных на валидацию (параметр test_size)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)

In [ ]:
# проверяем
test_data.shape, train_data.shape, X.shape, X_train.shape, X_test.shape

# Model 
Сам ML

In [ ]:
# Импортируем необходимые библиотеки:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели

In [ ]:
# Создаём модель (НАСТРОЙКИ НЕ ТРОГАЕМ)
model = RandomForestRegressor(n_estimators=100, verbose=1, n_jobs=-1, random_state=RANDOM_SEED)

In [ ]:
# Обучаем модель на тестовом наборе данных
model.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = model.predict(X_test)

In [ ]:
# Округлим предсказанные значения до степени округления целевой переменной
y_pred = np.round(y_pred*2)/2

In [ ]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))

In [ ]:
# в RandomForestRegressor есть возможность вывести самые важные признаки для модели
plt.rcParams['figure.figsize'] = (10,10)
feat_importances = pd.Series(model.feature_importances_, index=X.columns)
feat_importances.nlargest(10).plot(kind='barh')

# Submission
Если все устраевает - готовим Submission на кагл

In [ ]:
test_data.sample(10)

In [ ]:
test_data = test_data.drop(['Rating'], axis=1)

In [ ]:
sample_submission

In [ ]:
predict_submission = model.predict(test_data)

In [ ]:
predict_submission

In [ ]:
predict_submission = list(map(lambda x: round(x * 2)/2, predict_submission))
predict_submission

In [ ]:
sample_submission['Rating'] = predict_submission
sample_submission.to_csv('submission.csv', index=False)
sample_submission.head(10)